# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Segmento 5

In [42]:
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [43]:
df = pd.read_csv('Segm 5.csv')
df.head(2)

,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR,CORREO
0,HELADOS 2 LITROS,3200010002,CRUFI,424169,C,33913931.0,CI:3931,99361891,NaN
1,HELADOS 2 LITROS,3200010002,CRUFI,359467,C,33947099.0,CI:7099,99481357,NaN


In [44]:
tuple(df['SUB_NAME'].unique())

('HELADOS 2 LITROS',
 'YOGURES BEBIBLES ENTERO',
 'PACKS',
 'JABONES DETERGENTE LIQUIDOS PARA ROPA',
 'PAPAS FRITAS EN BOLSA',
 'PAPAS CONGELADAS',
 'PAPEL HIGIENICO HOJA DOBLE Y TRIPLE',
 'LECHE ESPECIALES',
 'CHOCOLATADA',
 'PANALES VALUE',
 'QUESO SANDWICH AL CORTE',
 'CERVEZAS NACIONALES',
 'JAMON COCIDO AL CORTE')

## A. Vinieron en 2024 a TATA

In [45]:
query = '''
SELECT
    DISTINCT FFM.SOCI_SOCI_ID
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
WHERE
    YEAR(FFM.TIEM_DIA_ID) = 2024
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
'''

cursor.execute(query)
vinieron_2024 = cursor.fetch_pandas_all()
vinieron_2024.head(2)

,SOCI_SOCI_ID
0,1258476
1,1589302


## B. Vinieron en 2024 y consumieron alguna de las SUBCLASE

In [46]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUB_NAME
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    SUB.SUB_NAME IN {subclases_snow}
    AND YEAR(FFM.TIEM_DIA_ID) = 2024
'''

cursor.execute(query.format(subclases_snow = tuple(df['SUB_NAME'].unique())))
vinieron_2024_2 = cursor.fetch_pandas_all()

vinieron_2024_2 = vinieron_2024_2[vinieron_2024_2['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]

vinieron_2024_2.head(2)

,TICKET,SOCI_SOCI_ID,SUB_NAME
20,202401083031150270,49028,YOGURES BEBIBLES ENTERO
35,20240108806953764,338318,YOGURES BEBIBLES ENTERO


In [47]:
resumen_vinieron_2024_2 = vinieron_2024_2.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_vinieron_2024_2.sort_values(by = 'SOCI_SOCI_ID', ascending = False)

,SUB_NAME,SOCI_SOCI_ID
12,YOGURES BEBIBLES ENTERO,582
9,PAPAS FRITAS EN BOLSA,456
1,CHOCOLATADA,428
4,JAMON COCIDO AL CORTE,406
6,PACKS,346
11,QUESO SANDWICH AL CORTE,286
3,JABONES DETERGENTE LIQUIDOS PARA ROPA,245
10,PAPEL HIGIENICO HOJA DOBLE Y TRIPLE,219
8,PAPAS CONGELADAS,215
0,CERVEZAS NACIONALES,186


In [48]:
int(round(vinieron_2024_2[vinieron_2024_2['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique() / df['SOCI_SOCI_ID'].nunique() * 100, 0))

4

In [49]:
df['SOCI_SOCI_ID'].nunique()
vinieron_2024[vinieron_2024['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique()

print(f"El Segmento 5 comprende clientes que en el ultimo mes no vengan a TATA y tengan Score 5 en la subclase. \nDe los {df['SOCI_SOCI_ID'].nunique()} clientes que cumplen dicho criterio, un {int(round(vinieron_2024[vinieron_2024['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique() / df['SOCI_SOCI_ID'].nunique() * 100, 0))}% volvieron a TATA en el 2024\nDe los mismos, un {int(round(vinieron_2024_2[vinieron_2024_2['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique() / df['SOCI_SOCI_ID'].nunique() * 100, 0))}% volvio en 2024 y consumio alguna de las SUBCLASES ofrecidas, siendo YOGURES BEBIBLES ENTERO, CHOCOLATADA , JAMON COCIDO AL CORTE y PAPAS FRITAS EN BOLSA las categorias mas demandadas")

El Segmento 5 comprende clientes que en el ultimo mes no vengan a TATA y tengan Score 5 en la subclase. 
De los 68557 clientes que cumplen dicho criterio, un 13% volvieron a TATA en el 2024
De los mismos, un 4% volvio en 2024 y consumio alguna de las SUBCLASES ofrecidas, siendo YOGURES BEBIBLES ENTERO, CHOCOLATADA , JAMON COCIDO AL CORTE y PAPAS FRITAS EN BOLSA las categorias mas demandadas


## Venta y GB1

In [50]:
query = '''
SELECT
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    YEAR(FFM.TIEM_DIA_ID) = 2024
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUB_NAME IN {subclases_query}
    AND FFM.SOCI_SOCI_ID IN {socios_query}
'''

cursor.execute(query.format(
    socios_query = tuple(vinieron_2024[vinieron_2024['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].unique()),
    subclases_query = tuple(df['SUB_NAME'].unique())
    ))
ventas_segm_5 = cursor.fetch_pandas_all()

ventas_segm_5

,VENTAS,GB1
0,584571.8,159816.881116


In [ ]:
584.571
159.816

# Segmento 3

In [ ]:
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [62]:
all_files = os.listdir()

files = [file for file in all_files if 'Segm 3.1' in file and file.endswith('.csv')]

df_2 = pd.DataFrame()

for file in files:
    df_aux = pd.read_csv(file)
    df_2 = pd.concat([df_aux, df_2], ignore_index=True)
print(df_2.shape[0])
df_2.head(2)

1442018


,SUB_NAME,SUBCLASE,MARCA,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,ULTIMOS_4,CELULAR,CORREO
0,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,116021,C,36817568,CI:7568,94346637.0,NaN
1,YOGURES BEBIBLES ENTERO,7900010002,BIOTOP,162888,C,36848050,CI:8050,99166458.0,NaN


In [63]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUB_NAME
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    SUB.SUB_NAME IN {subclases_snow}
    AND YEAR(FFM.TIEM_DIA_ID) = 2024
'''

cursor.execute(query.format(
    subclases_snow = tuple(df_2['SUB_NAME'].unique())
    ))
vinieron_2024_3 = cursor.fetch_pandas_all()

vinieron_2024_3 = vinieron_2024_3[vinieron_2024_3['SOCI_SOCI_ID'].isin(df_2['SOCI_SOCI_ID'])]

vinieron_2024_3.head(2)

,TICKET,SOCI_SOCI_ID,SUB_NAME
0,202401043481340497,342781,PAPAS CONGELADAS
1,202401043481340497,342781,PAPAS CONGELADAS


In [64]:
vinieron_2024_3['SOCI_SOCI_ID'].nunique()

73540

In [65]:
resumen_vinieron_2024_3 = vinieron_2024_3.groupby('SUB_NAME')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_vinieron_2024_3.sort_values(by = 'SOCI_SOCI_ID', ascending = False)

,SUB_NAME,SOCI_SOCI_ID
12,YOGURES BEBIBLES ENTERO,25108
4,JAMON COCIDO AL CORTE,16848
1,CHOCOLATADA,15861
9,PAPAS FRITAS EN BOLSA,13763
11,QUESO SANDWICH AL CORTE,11439
6,PACKS,10984
3,JABONES DETERGENTE LIQUIDOS PARA ROPA,8909
10,PAPEL HIGIENICO HOJA DOBLE Y TRIPLE,8112
8,PAPAS CONGELADAS,7180
0,CERVEZAS NACIONALES,6478


In [68]:
print(f"El Segmento 3 comprende clientes activos que no compran la subclase y tienen score 5. \nDe los {df_2['SOCI_SOCI_ID'].nunique()} clientes que cumplen dicho criterio, un {int(round(vinieron_2024_3[vinieron_2024_3['SOCI_SOCI_ID'].isin(df_2['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique() / df_2['SOCI_SOCI_ID'].nunique() * 100, 0))}% volvio en 2024 y consumio alguna de las SUBCLASES ofrecidas, siendo YOGURES BEBIBLES ENTERO, JAMON COCIDO AL CORTE, CHOCOLATADA y PAPAS FRITAS EN BOLSA las categorias mas demandadas")

El Segmento 3 comprende clientes activos que no compran la subclase y tienen score 5. 
De los 203698 clientes que cumplen dicho criterio, un 36% volvio en 2024 y consumio alguna de las SUBCLASES ofrecidas, siendo YOGURES BEBIBLES ENTERO, JAMON COCIDO AL CORTE, CHOCOLATADA y PAPAS FRITAS EN BOLSA las categorias mas demandadas


In [67]:
df_2['SOCI_SOCI_ID'].nunique()
#vinieron_2024_3[vinieron_2024_3['SOCI_SOCI_ID'].isin(df['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique()

203698

## Venta y GB1

In [71]:
query = '''
SELECT
    SUB.SUB_NAME,
    FFM.SOCI_SOCI_ID,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    YEAR(FFM.TIEM_DIA_ID) = 2024
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUB_NAME IN {subclases_query}
GROUP BY
    ALL
'''

cursor.execute(query.format(
    subclases_query = tuple(df_2['SUB_NAME'].unique())
    ))
ventas_segm_3 = cursor.fetch_pandas_all()
ventas_segm_3.head(2)

,SUB_NAME,SOCI_SOCI_ID,VENTAS,GB1
0,PAPAS FRITAS EN BOLSA,1736213,122.13,34.2537
1,PAPAS CONGELADAS,295351,734.43,263.4377


In [72]:
ventas_segm_3['SUB_NAME'].nunique()

13

In [78]:
ventas_segm_3_2 = pd.DataFrame()
for i in ventas_segm_3['SUB_NAME'].unique():
    print(i)
    clientes = tuple(df_2['SOCI_SOCI_ID'][df_2['SUB_NAME'] == i])
    df_aux = ventas_segm_3[(ventas_segm_3['SUB_NAME'] == i) & (ventas_segm_3['SOCI_SOCI_ID'].isin(clientes))]
    ventas_segm_3_2 = pd.concat([ventas_segm_3_2, df_aux])
print(f"Ventas --> {round(ventas_segm_3_2['VENTAS'].sum())}")
print(f"GB1 --> {round(ventas_segm_3_2['GB1'].sum())}")

PAPAS FRITAS EN BOLSA
PAPAS CONGELADAS
YOGURES BEBIBLES ENTERO
JAMON COCIDO AL CORTE
QUESO SANDWICH AL CORTE
PAPEL HIGIENICO HOJA DOBLE Y TRIPLE
PACKS
LECHE ESPECIALES
CERVEZAS NACIONALES
HELADOS 2 LITROS
CHOCOLATADA
JABONES DETERGENTE LIQUIDOS PARA ROPA
PANALES VALUE
Ventas --> 7439696
GB1 --> 2616892


# Fin